In [1]:
suppressPackageStartupMessages(library(readxl))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(patchwork))
suppressPackageStartupMessages(library(pheatmap))
suppressPackageStartupMessages(library(stringr))
library(hise)
library(plyr)
library(purrr)

------------------------------------------------------------------------------

You have loaded plyr after dplyr - this is likely to cause problems.
If you need functions from both plyr and dplyr, please load plyr first, then dplyr:
library(plyr); library(dplyr)

------------------------------------------------------------------------------


Attaching package: ‘plyr’


The following objects are masked from ‘package:dplyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize



Attaching package: ‘purrr’


The following object is masked from ‘package:plyr’:

    compact




In [48]:
fileDescToDataframe_beta <- function(descriptors,keep_labs = FALSE) {
  descriptors<-descriptors[[1]]
  assertthat::assert_that(typeof(descriptors) == "list")
  assertthat::assert_that(typeof(keep_labs) == "logical")
  
  do.call(
    rbind.fill,
    lapply(
      descriptors,
      function(desc) {
        desc <- unlist(desc)
        desc <- desc[!grepl("scheme", names(desc))]
        names(desc) <- sub("^descriptors.","",names(desc))
        desc <- desc[!grepl("^specimens", names(desc))]
        desc <- desc[!grepl("^lab", names(desc))]
        desc <- desc[!grepl("^emr", names(desc))]
        desc <- desc[!grepl("^survey", names(desc))]
        desc <- desc[!grepl("^revision", names(desc))]
        desc <- desc[!grepl("^file.userTags", names(desc))]
        desc <- as.list(desc)
        df <- as.data.frame(desc)
        df
      }
    )
  )
}

fileDescToDataframe <- function(descriptors,
                                keep_labs = FALSE) {
  
  assertthat::assert_that(typeof(descriptors) == "list")
  assertthat::assert_that(typeof(keep_labs) == "logical")
  
  do.call(
    rbind,
    lapply(
      descriptors,
      function(desc) {
        desc <- unlist(desc)
        desc <- desc[!grepl("scheme", names(desc))]
        names(desc) <- sub("^descriptors.","",names(desc))
        desc <- desc[!grepl("^specimens", names(desc))]
        desc <- desc[!grepl("^lab", names(desc))]
        desc <- desc[!grepl("^emr", names(desc))]
        desc <- desc[!grepl("^survey", names(desc))]
        desc <- desc[!grepl("^revision", names(desc))]
        desc <- desc[!grepl("^file.userTags", names(desc))]
        desc <- as.list(desc)
        df <- as.data.frame(desc)
        df
      }
    )
  )
}

In [49]:
BR1_rna_desc <- getFileDescriptors(
    fileType = "scRNA-seq-labeled", 
    filter = list(cohort.cohortGuid = "BR1"))
BR2_rna_desc <- getFileDescriptors(
    fileType = "scRNA-seq-labeled", 
    filter = list(cohort.cohortGuid = "BR2"))

[1] "Authorization token invalid or expired."
[1] "Retrying..."


In [50]:
BR1_rna_desc <- fileDescToDataframe_beta(BR1_rna_desc)
BR2_rna_desc <- fileDescToDataframe_beta(BR2_rna_desc)

In [51]:
meta_data<-rbind.fill(BR1_rna_desc , BR2_rna_desc )

In [52]:
meta_data<-meta_data%>% filter(!file.batchID=="B004")%>% filter(!subject.subjectGuid%in% c("BR2007","BR2049",'BR1034'))

In [53]:
meta_data$pbmc_sample_id<-gsub("_","",paste0("PB0",substr(sub(".*PB0", "", meta_data$file.name),1,8)))
meta_data<-meta_data %>% filter(!grepl("EXP",file.batchID))%>% arrange(pbmc_sample_id)
meta_data<-meta_data[!duplicated(meta_data[, c("sample.sampleKitGuid")], fromLast=T),]%>% arrange(sample.sampleKitGuid)
meta_data<-meta_data%>%mutate(sample.visitName=ifelse(sample.visitName=="Other - Non-Flu",sample.visitDetails,sample.visitName) )
write.csv(meta_data,paste0("hise_meta_data_",Sys.Date(),".csv"))

In [8]:
meta_data_Y1D0<-meta_data %>% filter(sample.visitName=='Flu Year 1 Day 0')
table(meta_data_Y1D0$cohort.cohortGuid)


BR1 BR2 
 47  45 

In [9]:
meta_data_non_Y1D0<-meta_data %>% filter(sample.visitName!='Flu Year 1 Day 0')
table(meta_data_non_Y1D0$cohort.cohortGuid)


BR1 BR2 
371 405 

In [14]:
table(meta_data_non_Y1D0$sample.visitName)


       Flu Year 1 Day 7       Flu Year 1 Day 90  Flu Year 1 Stand-Alone 
                     92                      89                      14 
       Flu Year 2 Day 0        Flu Year 2 Day 7       Flu Year 2 Day 90 
                     84                      84                      82 
 Flu Year 2 Stand-Alone  Flu Year 3 Stand-Alone  Immune Variation Day 0 
                     22                      47                      89 
 Immune Variation Day 7 Immune Variation Day 90 
                     89                      84 

In [37]:
folder_names <- list.dirs(path = 'cache', full.names = FALSE, recursive = TRUE)
meta_data_scRNA_list<-list()
for (i in 1:dim(meta_data_non_Y1D0)[1]){
if (meta_data_non_Y1D0$file.id[i] %in% folder_names){
next
}
meta_data_scRNA_single <- cacheFiles(list(meta_data_non_Y1D0$file.id[i]))
print (i)
}

submitting request as query ID first...

retrieving files using fileIDS...



[1] "Initiating file download for B072-P1_PB01565-01_2021-07-12T17:24:36.579193615Z_labeled.h5"
[1] "Download successful."
[1] 356


In [38]:
files<-data.frame(list.files(path = "cache/", pattern = 'h5', all.files = TRUE,
           full.names = FALSE, recursive = TRUE,
           ignore.case = FALSE, include.dirs = FALSE, no.. = FALSE))

In [39]:
length(files)

[1] 1

In [40]:
colnames(files)<-'file.path'

In [41]:
separated <- strsplit(files$file.path, split = '/')

In [42]:

# Use strsplit to separate each string on '/', then unlist and rbind to create a matrix
separated_matrix <- do.call(rbind, lapply(files$file.path, function(x) unlist(strsplit(x, split = '/'))))

# Convert the matrix to a data frame
df <- as.data.frame(separated_matrix, stringsAsFactors = FALSE)

# Set column names
colnames(df) <- c("file.id", "file.name.downloaded")


In [44]:
meta_data_non_Y1D0<-left_join(meta_data_non_Y1D0,cbind(df,files),by=('file.id'))

In [45]:
meta_data_non_Y1D0$file.path<-paste0('/home/jupyter/BRI_Analysis/scRNA/cache/',meta_data_non_Y1D0$file.path)

In [46]:
write.csv(meta_data_non_Y1D0,paste0("hise_meta_data_",Sys.Date(),"_nonY1D0.csv"))